# DCSI 592: Capstone II

## Predicting Crop Yield in Colorado: Modeling

Dataset: https://catalog.data.gov/dataset/usda-ars-colorado-maize-water-productivity-dataset-2012-2013-f9f68


## 0.0 Imports, Helper Functions & Loading Data

In [68]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

In [33]:
# apply log transformation and add a constant to deal with 0s
def apply_log_transform_with_constant(dataframe, constant=1e-5):
    # Copy the input dataframe to avoid modifying the original data
    transformed_df = dataframe.copy()
        # Apply logarithmic transformation to numeric columns
    for col in transformed_df.select_dtypes(include=['number']).columns:
        # Add a small constant value to avoid zero values
        transformed_df[col] = np.log(transformed_df[col] + constant)
    return transformed_df

# organizes and neatly displays model results for model evaluation
def display_model_results(train_mse, train_r2, test_mse, test_r2, cross_val_rmse, cross_val_r2, model_description):
    print(f"Results for {model_description.upper()}:")
    print(f"Training MSE: {train_mse:.4f}")
    print(f"Training R^2: {train_r2:.4f}")
    print(f"Test MSE: {test_mse:.4f}")
    print(f"Test R^2: {test_r2:.4f}")
    print(f"Cross-Validation RMSE: {cross_val_rmse.mean():.4f} ± {cross_val_rmse.std():.4f}")
    print(f"Cross-Validation R^2: {cross_val_r2.mean():.4f} ± {cross_val_r2.std():.4f}")
    print("="*40)


# applies pipeline (feature scaling), desired algorithm, cross validation method, and evaluates performance
def evaluate_pipeline(pipeline, X_train, y_train, X_test, y_test, model_description, cv):
    # train the model
    pipeline.fit(X_train, y_train)
    
    # calculate training MSE and R^2
    y_train_pred = pipeline.predict(X_train)
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    
    # Calculate test MSE and R^2
    y_test_pred = pipeline.predict(X_test)
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # perform cross validation on training set
    cross_val_mse = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
    cross_val_rmse = np.sqrt(-cross_val_mse)
    
    # cross validation R^2 score
    cross_val_r2 = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='r2')
    
    # display model results using `display_model_results()` 
    display_model_results(train_mse, train_r2, test_mse, test_r2, cross_val_rmse, cross_val_r2, model_description)

    return


In [34]:
FILE = 'cleaned_annual_data.csv'
maize_data = pd.read_csv(FILE)
maize_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Grain Yield_15.5%mc_Kg ha-1  94 non-null     float64
 1   Max_LAI                      94 non-null     float64
 2   Annual_ETc mm                94 non-null     float64
 3   Trt_code                     94 non-null     int64  
 4   Plant density plants ha-1    94 non-null     float64
 5   HI                           94 non-null     float64
dtypes: float64(5), int64(1)
memory usage: 4.5 KB


## 0.1 Features

For our model, we will predict `grain yield` using: 
- irrigation `treatment code` (numerical value that represents the watering deprivation treatment used: the lower value treatments (1, 2, 3...) received more water, while higher values (9, 10, 11...) got less during both periods.Table in appendix)
- `plant density`
- `max leaf area index`

Concept: if a farmer was facing a drought (limited water supply), could they use a model to optimize their watering and still achieve grain yield goals?

In [35]:
# corrected variable name 
cols_for_model = ["Grain Yield_15.5%mc_Kg ha-1", "Trt_code","Plant density plants ha-1", "Max_LAI"]
maize_model_data = maize_data[cols_for_model]
maize_model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Grain Yield_15.5%mc_Kg ha-1  94 non-null     float64
 1   Trt_code                     94 non-null     int64  
 2   Plant density plants ha-1    94 non-null     float64
 3   Max_LAI                      94 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 3.1 KB


## 0.2 Log Transformation

In [36]:
log_transformed_model_data = apply_log_transform_with_constant(maize_model_data)
log_transformed_model_data.head()

,Grain Yield_15.5%mc_Kg ha-1,Trt_code,Plant density plants ha-1,Max_LAI
0,9.744325,0.000010,11.343192,1.489657
1,9.750410,0.000010,11.288274,1.575948
2,9.565569,0.000010,11.265414,1.461266
3,9.578622,0.000010,11.258034,1.429405
4,9.429054,0.693152,11.287757,1.646937


# Modeling
## 1. Linear Regression Results
### 1.0 Linear Regression: Baseline (no scaling/no log transformation)

In [41]:
# Define features as `X` and target variable (Grain Yield) as `y`
X = maize_model_data.drop(columns=['Grain Yield_15.5%mc_Kg ha-1'])
y = maize_model_data['Grain Yield_15.5%mc_Kg ha-1']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# initialize the model
linear_regression_model = LinearRegression()

# train the model on non-transformed data
linear_regression_model.fit(X_train, y_train)

# calculate training MSE and R^2
y_train_pred = linear_regression_model.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# calculate test MSE and R^2
y_test_pred = linear_regression_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Perform cross-validation on the training set
cv = KFold(n_splits=5, random_state=42, shuffle=True)
cross_val_mse = cross_val_score(linear_regression_model, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
cross_val_rmse = np.sqrt(-cross_val_mse)

# Perform cross-validation for R^2 score
cross_val_r2 = cross_val_score(linear_regression_model, X_train, y_train, cv=cv, scoring='r2')

# Display the results
display_model_results(train_mse, train_r2, test_mse, test_r2, cross_val_rmse, cross_val_r2, model_description="Linear Regression (No Scaling / No Log Transformation)")

Results for LINEAR REGRESSION (NO SCALING / NO LOG TRANSFORMATION):
Training MSE: 2184176.2584
Training R^2: 0.3909
Test MSE: 3737592.9884
Test R^2: 0.3870
Cross-Validation RMSE: 1547.5232 ± 265.7279
Cross-Validation R^2: 0.2609 ± 0.2117


In [42]:
# Define features as `X` and target variable (Grain Yield) as `y`
X = log_transformed_model_data.drop(columns=['Grain Yield_15.5%mc_Kg ha-1'])
y = log_transformed_model_data['Grain Yield_15.5%mc_Kg ha-1']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# initialize the model
linear_regression_model = LinearRegression()

# train the model on non-transformed data
linear_regression_model.fit(X_train, y_train)

# calculate training MSE and R^2
y_train_pred = linear_regression_model.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# calculate test MSE and R^2
y_test_pred = linear_regression_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Perform cross-validation on the training set
cv = KFold(n_splits=5, random_state=42, shuffle=True)
cross_val_mse = cross_val_score(linear_regression_model, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
cross_val_rmse = np.sqrt(-cross_val_mse)

# Perform cross-validation for R^2 score
cross_val_r2 = cross_val_score(linear_regression_model, X_train, y_train, cv=cv, scoring='r2')

# Display the results
display_model_results(train_mse, train_r2, test_mse, test_r2, cross_val_rmse, cross_val_r2, model_description="Linear Regression (Log Transformed / No Scaling)")

Results for LINEAR REGRESSION (LOG TRANSFORMED / NO SCALING):
Training MSE: 0.0159
Training R^2: 0.3242
Test MSE: 0.0234
Test R^2: 0.3933
Cross-Validation RMSE: 0.1319 ± 0.0262
Cross-Validation R^2: 0.1849 ± 0.2005


### 1.1 Linear Regression: MinMax & Standard Scaling

In [59]:
# define features as `X` and target variable as `y`   
X = maize_model_data.drop(columns=['Grain Yield_15.5%mc_Kg ha-1'])
y = maize_model_data['Grain Yield_15.5%mc_Kg ha-1']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# standard scaling, linear regression
linear_pipeline_std_scaling = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])

# minmax scaling, linear regression
linear_pipeline_minmax_scaling = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', LinearRegression())
])

# instantiate cross validation
cv = KFold(n_splits=5, random_state=42, shuffle=True)

evaluate_pipeline(linear_pipeline_std_scaling, X_train, y_train, X_test, y_test, model_description="Linear Regression (Std Scaling / No Log Trans.)", cv=cv)
evaluate_pipeline(linear_pipeline_minmax_scaling, X_train, y_train, X_test, y_test, model_description="Linear Regression (MinMax Scaling / No Log Trans.)", cv=cv)

Results for LINEAR REGRESSION (STD SCALING / NO LOG TRANS.):
Training MSE: 2184176.2584
Training R^2: 0.3909
Test MSE: 3737592.9884
Test R^2: 0.3870
Cross-Validation RMSE: 1547.5232 ± 265.7279
Cross-Validation R^2: 0.2609 ± 0.2117
Results for LINEAR REGRESSION (MINMAX SCALING / NO LOG TRANS.):
Training MSE: 2184176.2584
Training R^2: 0.3909
Test MSE: 3737592.9884
Test R^2: 0.3870
Cross-Validation RMSE: 1547.5232 ± 265.7279
Cross-Validation R^2: 0.2609 ± 0.2117


MinMax & Standard Scaling (Log Transformed)



In [58]:
# define features as `X` and target variable as `y`   
X = log_transformed_model_data.drop(columns=['Grain Yield_15.5%mc_Kg ha-1'])
y = log_transformed_model_data['Grain Yield_15.5%mc_Kg ha-1']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# instantiate cross validation
cv = KFold(n_splits=5, random_state=42, shuffle=True)

# standard scaling, linear regression
linear_pipeline_std_scaling = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])

# minmax scaling, linear regression
linear_pipeline_minmax_scaling = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', LinearRegression())
])

evaluate_pipeline(linear_pipeline_std_scaling, X_train, y_train, X_test, y_test, model_description="Linear Regression: Log Transformed (Std Scaling)", cv=cv)
evaluate_pipeline(linear_pipeline_minmax_scaling, X_train, y_train, X_test, y_test, model_description="Linear Regression: Log Transformed (MinMax Scaling)", cv=cv)

Results for LINEAR REGRESSION: LOG TRANSFORMED (STD SCALING):
Training MSE: 0.0159
Training R^2: 0.3242
Test MSE: 0.0234
Test R^2: 0.3933
Cross-Validation RMSE: 0.1319 ± 0.0262
Cross-Validation R^2: 0.1849 ± 0.2005
Results for LINEAR REGRESSION: LOG TRANSFORMED (MINMAX SCALING):
Training MSE: 0.0159
Training R^2: 0.3242
Test MSE: 0.0234
Test R^2: 0.3933
Cross-Validation RMSE: 0.1319 ± 0.0262
Cross-Validation R^2: 0.1849 ± 0.2005


# 2.0 Ridge Regression 
<br> Log transformation with two feature scaling techniques: standard and min-max 




In [57]:
# define features as `X` and target variable as `y`   
X = log_transformed_model_data.drop(columns=['Grain Yield_15.5%mc_Kg ha-1'])
y = log_transformed_model_data['Grain Yield_15.5%mc_Kg ha-1']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# instantiate cross validation
cv = KFold(n_splits=5, random_state=42, shuffle=True)

# standard scaling, linear regression
ridge_pipeline_std_scaling = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Ridge())
])

# minmax scaling, linear regression
ridge_pipeline_minmax_scaling = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', Ridge())
])

evaluate_pipeline(ridge_pipeline_std_scaling, X_train, y_train, X_test, y_test, model_description="Ridge Regression: Log Transformed (Std Scaling)", cv=cv)
evaluate_pipeline(ridge_pipeline_minmax_scaling, X_train, y_train, X_test, y_test, model_description="Ridge Regression: Log Transformed (MinMax Scaling)", cv=cv)

Results for RIDGE REGRESSION: LOG TRANSFORMED (STD SCALING):
Training MSE: 0.0159
Training R^2: 0.3242
Test MSE: 0.0235
Test R^2: 0.3914
Cross-Validation RMSE: 0.1318 ± 0.0263
Cross-Validation R^2: 0.1868 ± 0.1979
Results for RIDGE REGRESSION: LOG TRANSFORMED (MINMAX SCALING):
Training MSE: 0.0161
Training R^2: 0.3168
Test MSE: 0.0246
Test R^2: 0.3621
Cross-Validation RMSE: 0.1312 ± 0.0268
Cross-Validation R^2: 0.2007 ± 0.1693


# 3.0 Random Forest 
Log transformed data with two different scaling techniques

In [60]:
# define features as `X` and target variable as `y`   
y = log_transformed_model_data['Grain Yield_15.5%mc_Kg ha-1']
X = log_transformed_model_data.drop(columns=['Grain Yield_15.5%mc_Kg ha-1'])

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# instantiate cross validation
cv = KFold(n_splits=5, random_state=42, shuffle=True)

# apply RandomForest with standard scaling
random_forest_std_scaler_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('model', RandomForestRegressor())
])

# apply RandomForest with min max scaling
random_forest_minmax_scaler_pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', RandomForestRegressor())
])


evaluate_pipeline(random_forest_std_scaler_pipeline, X_train, y_train, X_test, y_test, model_description="Random Forest: Std Scaling", cv=cv)
evaluate_pipeline(random_forest_minmax_scaler_pipeline, X_train, y_train, X_test, y_test, model_description="Random Forest: MinMax Scaling", cv=cv)


Results for RANDOM FOREST: STD SCALING:
Training MSE: 0.0024
Training R^2: 0.8988
Test MSE: 0.0195
Test R^2: 0.4945
Cross-Validation RMSE: 0.1204 ± 0.0146
Cross-Validation R^2: 0.3265 ± 0.1382
Results for RANDOM FOREST: MINMAX SCALING:
Training MSE: 0.0024
Training R^2: 0.8992
Test MSE: 0.0182
Test R^2: 0.5283
Cross-Validation RMSE: 0.1163 ± 0.0146
Cross-Validation R^2: 0.3185 ± 0.1551


## 3.1 Random Forest - Hyperparameter Tuning 

In [76]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Define features as `X` and target variable as `y`
X = log_transformed_model_data.drop(columns=['Grain Yield_15.5%mc_Kg ha-1'])
y = log_transformed_model_data['Grain Yield_15.5%mc_Kg ha-1']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter distribution for RandomizedSearchCV
param_dist = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [10, 20, 30, None],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__max_features': ['sqrt', 'log2', None]
}

# Create a pipeline that includes standard scaling and the random forest model
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor(random_state=42))
])

# Initialize RandomizedSearchCV with the pipeline
random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist, 
                                   n_iter=50, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', 
                                   random_state=42, verbose=2)

# Perform the random search
random_search.fit(X_train, y_train)

# Print the best parameters
print(f"Best parameters found by RandomizedSearchCV: {random_search.best_params_}\n")

# Get the best estimator (pipeline with the best model)
best_pipeline = random_search.best_estimator_

# Calculate training MSE and R^2
y_train_pred = best_pipeline.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Calculate test MSE and R^2
y_test_pred = best_pipeline.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Perform cross-validation on the training set
cv = KFold(n_splits=5, random_state=42, shuffle=True)
cross_val_mse = cross_val_score(best_pipeline, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
cross_val_rmse = np.sqrt(-cross_val_mse)

# Cross-validation R^2 score
cross_val_r2 = cross_val_score(best_pipeline, X_train, y_train, cv=cv, scoring='r2')

# Display model results
display_model_results(train_mse, train_r2, test_mse, test_r2, cross_val_rmse, cross_val_r2, model_description="Tuned Random Forest Regression with Standard Scaling")

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found by RandomizedSearchCV: {'model__n_estimators': 200, 'model__min_samples_split': 10, 'model__min_samples_leaf': 2, 'model__max_features': None, 'model__max_depth': None}

Results for TUNED RANDOM FOREST REGRESSION WITH STANDARD SCALING:
Training MSE: 0.0077
Training R^2: 0.6737
Test MSE: 0.0188
Test R^2: 0.5145
Cross-Validation RMSE: 0.1174 ± 0.0148
Cross-Validation R^2: 0.3464 ± 0.1162


# 4.0 Gradient Boosting Regression
Log transformed, looking at two scaling techniques (std and minmax)

In [77]:
# define features as `X` and target variable as `y`   
X = log_transformed_model_data.drop(columns=['Grain Yield_15.5%mc_Kg ha-1'])
y = log_transformed_model_data['Grain Yield_15.5%mc_Kg ha-1']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# instantiate cross validation
cv = KFold(n_splits=5, random_state=42, shuffle=True)

# apply RandomForest with standard scaling
gradient_boosting_std_scaler_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('model', GradientBoostingRegressor())
])

# apply RandomForest with min max scaling
gradient_boosting_minmax_scaler_pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', GradientBoostingRegressor())
])


evaluate_pipeline(gradient_boosting_std_scaler_pipeline, X_train, y_train, X_test, y_test, model_description="Gradient Boosting: Std Scaling", cv=cv)
evaluate_pipeline(gradient_boosting_minmax_scaler_pipeline, X_train, y_train, X_test, y_test, model_description="Gradient Boosting: MinMax Scaling", cv=cv)


Results for GRADIENT BOOSTING: STD SCALING:
Training MSE: 0.0006
Training R^2: 0.9758
Test MSE: 0.0197
Test R^2: 0.4903
Cross-Validation RMSE: 0.1331 ± 0.0194
Cross-Validation R^2: 0.1050 ± 0.3266
Results for GRADIENT BOOSTING: MINMAX SCALING:
Training MSE: 0.0006
Training R^2: 0.9758
Test MSE: 0.0197
Test R^2: 0.4904
Cross-Validation RMSE: 0.1332 ± 0.0186
Cross-Validation R^2: 0.1014 ± 0.3373


# 5.0 SVR with RBF & Poly kernel and scaling comparisons

In [49]:
# define features as `X` and target variable as `y`   
X = log_transformed_model_data.drop(columns=['Grain Yield_15.5%mc_Kg ha-1'])
y = log_transformed_model_data['Grain Yield_15.5%mc_Kg ha-1']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# instantiate cross validation
cv = KFold(n_splits=5, random_state=42, shuffle=True)

# apply SVR (RBF Kernel) with standard scaling
svr_std_scaler_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('model', SVR(kernel='rbf'))
])

# apply SVR (RBF Kernel) with min max scaling
svr_minmax_scaler_pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', SVR(kernel='rbf'))
])

# apply SVR (Poly Kernel) with standard scaling
svr_poly_std_scaler_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('model', SVR(kernel='poly'))
])

# apply SVR (Poly Kernel) with min max scaling
svr_poly_minmax_scaler_pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', SVR(kernel='poly'))
])

evaluate_pipeline(svr_std_scaler_pipeline, X_train, y_train, X_test, y_test, model_description="SVR (RBF Kernel): Std Scaling", cv=cv)
evaluate_pipeline(svr_minmax_scaler_pipeline, X_train, y_train, X_test, y_test, model_description="SVR (RBF Kernel): MinMax Scaling", cv=cv)
evaluate_pipeline(svr_poly_std_scaler_pipeline, X_train, y_train, X_test, y_test, model_description="SVR (Poly Kernel): Std Scaling", cv=cv)
evaluate_pipeline(svr_poly_minmax_scaler_pipeline, X_train, y_train, X_test, y_test, model_description="SVR (Poly Kernel): MinMax Scaling", cv=cv)

Results for SVR (RBF KERNEL): STD SCALING:
Training MSE: 0.0107
Training R^2: 0.5451
Test MSE: 0.0221
Test R^2: 0.4270
Cross-Validation RMSE: 0.1362 ± 0.0249
Cross-Validation R^2: 0.0994 ± 0.2982
Results for SVR (RBF KERNEL): MINMAX SCALING:
Training MSE: 0.0112
Training R^2: 0.5225
Test MSE: 0.0219
Test R^2: 0.4319
Cross-Validation RMSE: 0.1282 ± 0.0221
Cross-Validation R^2: 0.2045 ± 0.2458
Results for SVR (POLY KERNEL): STD SCALING:
Training MSE: 0.0159
Training R^2: 0.3258
Test MSE: 0.0249
Test R^2: 0.3560
Cross-Validation RMSE: 0.1790 ± 0.0794
Cross-Validation R^2: -0.7383 ± 1.6082
Results for SVR (POLY KERNEL): MINMAX SCALING:
Training MSE: 0.0128
Training R^2: 0.4574
Test MSE: 0.0281
Test R^2: 0.2715
Cross-Validation RMSE: 0.1282 ± 0.0235
Cross-Validation R^2: 0.2034 ± 0.2722


# Results Summary

| Model                     | Preprocessing            | Feature Scaling | Target Scaling | Train MSE  | Test MSE  | Train R² | Test R² | CV RMSE   | CV RMSE STD | CV R²   | CV R² STD |
|---------------------------|--------------------------|-----------------|----------------|------------|-----------|----------|---------|------------|-------------|---------|-----------|
| Linear Regression          | None                     | None            | None           | 2184176.258 | 3737592.988 | 0.3909   | 0.387   | 1547.5232 | 265.7279    | 0.2609  | 0.2117    |
| Linear Regression          | None                     | Standard        | None           | 2184176.258 | 3737592.988 | 0.3909   | 0.387   | 1547.5232 | 265.7279    | 0.2609  | 0.2117    |
| Linear Regression          | None                     | MinMax          | None           | 2184176.258 | 3737592.988 | 0.3909   | 0.387   | 1547.5232 | 265.7279    | 0.2609  | 0.2117    |
| Linear Regression          | Log transformation       | Standard        | None           | 0.0159     | 0.0234    | 0.3242   | 0.3933  | 0.1319    | 0.0262      | 0.1849  | 0.2005    |
| Linear Regression          | Log transformation       | MinMax          | None           | 0.0159     | 0.0234    | 0.3242   | 0.3933  | 0.1319    | 0.0262      | 0.1849  | 0.2005    |
| Ridge Regression           | Log transformation       | Standard        | None           | 0.0159     | 0.0235    | 0.3242   | 0.3914  | 0.1318    | 0.0263      | 0.1868  | 0.1979    |
| Ridge Regression           | Log transformation       | MinMax          | None           | 0.0161     | 0.0246    | 0.3168   | 0.3621  | 0.1312    | 0.0268      | 0.2007  | 0.1693    |
| Random Forest              | Log transformation       | Standard        | None           | 0.0024     | 0.0195    | 0.8988   | 0.4945  | 0.1204    | 0.0146      | 0.3265  | 0.1382    |
| Random Forest              | Log transformation       | MinMax          | None           | 0.0024     | 0.0182    | 0.8992   | 0.5283  | 0.1163    | 0.0146      | 0.3185  | 0.1551    |
| Random Forest Tuned        | Log transformation       | Standard        | None           | 0.0077     | 0.0188    | 0.6737   | 0.5145  | 0.1174    | 0.0148      | 0.3464  | 0.1162    |
| *Random Forest RFE_3        | Log transformation       | None            | None           | 0.0179     | 0.0179    | 0.5364   | 0.5364  | 0.1202    | 0.0122      | 0.3068  | 0.136     |
| *Random Forest RFE_5        | Log transformation       | None            | None           | 0.0197     | 0.0197    | 0.4905   | 0.4905  | 0.1191    | 0.0095      | 0.3094  | 0.1636    |
| Gradient Boosting          | Log transformation       | Standard        | None           | 0.0006     | 0.0197    | 0.9758   | 0.4903  | 0.1331    | 0.0194      | 0.105   | 0.3266    |
| Gradient Boosting          | Log transformation       | MinMax          | None           | 0.0006     | 0.0197    | 0.9758   | 0.4904  | 0.1332    | 0.0186      | 0.1014  | 0.3373    |
| SVR with RBF kernel        | Log transformation       | Standard        | None           | 0.0107     | 0.0221    | 0.5451   | 0.427   | 0.1362    | 0.0249      | 0.0994  | 0.2982    |
| SVR with RBF kernel        | Log transformation       | MinMax          | None           | 0.0112     | 0.0219    | 0.5225   | 0.4319  | 0.1282    | 0.0221      | 0.2045  | 0.2458    |
| SVR with poly kernel       | Log transformation       | Standard        | None           | 0.0159     | 0.0249    | 0.3258   | 0.356   | 0.179     | 0.0794      | -0.7383 | 1.6082    |
| SVR with poly kernel       | Log transformation       | MinMax          | None           | 0.0128     | 0.0281    | 0.4574   | 0.2715  | 0.1282    | 0.0235      | 0.2034  | 0.2722    |

(*) See section after next steps / future work for this code.

# Next Steps / Future Work
- visualize modeling results 
- finish paper 
- continue exploring feature egineering: started with Recursive Feature Elimination for RandomForest (shown below) but need to clean up code and read more on this...
- future work: investigate if scaling target affects model performance 

### Random Forest Recursive Feature Elimination (RFE)

In [88]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# we want the log transformed data but with all cols to see if RFE can find better features and improve the model
log_data_all_cols = apply_log_transform_with_constant(maize_data)

X = log_data_all_cols.drop(columns=['Grain Yield_15.5%mc_Kg ha-1'])
y = log_data_all_cols['Grain Yield_15.5%mc_Kg ha-1']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize simple linear reg. model for RFE 
model = LinearRegression()

# apply RFE to select top features
rfe = RFE(estimator=model, n_features_to_select=5)  
X_train_rfe = rfe.fit_transform(X_train, y_train)
X_test_rfe = rfe.transform(X_test)

# display selected features
selected_features = X.columns[rfe.support_]
print(f"Selected Features: {selected_features}")

# Initialize the RandomForestRegressor model
rf_model = RandomForestRegressor(random_state=42)

# Train the model on the selected features
rf_model.fit(X_train_rfe, y_train)

# Make predictions on the test set
y_test_pred = rf_model.predict(X_test_rfe)

# Evaluate the model
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Perform cross-validation on the training set
cv = KFold(n_splits=5, random_state=42, shuffle=True)
cross_val_mse = cross_val_score(rf_model, X_train_rfe, y_train, cv=cv, scoring='neg_mean_squared_error')
cross_val_rmse = np.sqrt(-cross_val_mse)

# Cross-validation R^2 score
cross_val_r2 = cross_val_score(rf_model, X_train_rfe, y_train, cv=cv, scoring='r2')

# get results
display_model_results(test_mse, test_r2, test_mse, test_r2, cross_val_rmse, cross_val_r2, model_description="Random Forest Regression with RFE")

Selected Features: Index(['Max_LAI', 'Annual_ETc mm', 'Trt_code', 'Plant density plants ha-1',
       'HI'],
      dtype='object')
Results for RANDOM FOREST REGRESSION WITH RFE:
Training MSE: 0.0197
Training R^2: 0.4905
Test MSE: 0.0197
Test R^2: 0.4905
Cross-Validation RMSE: 0.1191 ± 0.0095
Cross-Validation R^2: 0.3094 ± 0.1636


In [87]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# what if we limit it to the features we identified ? can we get better results?
log_data_trimmed = apply_log_transform_with_constant(maize_model_data)

X = log_data_trimmed.drop(columns=['Grain Yield_15.5%mc_Kg ha-1'])
y = log_data_trimmed['Grain Yield_15.5%mc_Kg ha-1']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize simple linear reg. model for RFE 
model = LinearRegression()

# apply RFE to select top features
rfe = RFE(estimator=model, n_features_to_select=3)  
X_train_rfe = rfe.fit_transform(X_train, y_train)
X_test_rfe = rfe.transform(X_test)

# display selected features
selected_features = X.columns[rfe.support_]
print(f"Selected Features: {selected_features}")

# Initialize the RandomForestRegressor model
rf_model = RandomForestRegressor(random_state=42)

# Train the model on the selected features
rf_model.fit(X_train_rfe, y_train)

# Make predictions on the test set
y_test_pred = rf_model.predict(X_test_rfe)

# Evaluate the model
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Perform cross-validation on the training set
cv = KFold(n_splits=5, random_state=42, shuffle=True)
cross_val_mse = cross_val_score(rf_model, X_train_rfe, y_train, cv=cv, scoring='neg_mean_squared_error')
cross_val_rmse = np.sqrt(-cross_val_mse)

# Cross-validation R^2 score
cross_val_r2 = cross_val_score(rf_model, X_train_rfe, y_train, cv=cv, scoring='r2')

# get results
display_model_results(test_mse, test_r2, test_mse, test_r2, cross_val_rmse, cross_val_r2, model_description="Random Forest Regression with RFE")

Selected Features: Index(['Trt_code', 'Plant density plants ha-1', 'Max_LAI'], dtype='object')
Results for RANDOM FOREST REGRESSION WITH RFE:
Training MSE: 0.0179
Training R^2: 0.5364
Test MSE: 0.0179
Test R^2: 0.5364
Cross-Validation RMSE: 0.1202 ± 0.0122
Cross-Validation R^2: 0.3068 ± 0.1360


In [ ]:
SUMMARY = 'cleaned_annual_data.csv'
maize_data = pd.read_csv(FILE)
maize_data.info()

___________________________

# Appendix



| **Trt Code** | **Irr Treatment (late veg/grain filling)** |
|--------------|-------------------|
| 1            | 100/100           |
| 2            | 100/50            |
| 3            | 80/80             |
| 4            | 80/65             |
| 5            | 80/50             |
| 6            | 80/40             |
| 7            | 65/80             |
| 8            | 65/65             |
| 9            | 65/50             |
| 10           | 65/40             |
| 11           | 50/50             |
| 12           | 40/40             |

### Looking at model coefficients -- Ordered from most to least significant impact on grain yield: Treatment Code, Max Leaf Area Index, and Plant Density 

The coefficients for this model tell us:
- `Trt_code`: Strong negative relationship with grain yield. For every 1 unit increase in `Treatment_code` (which reduces water usage), the grain yield is expected to decrease by ~338.83 Kg / h1 units. (~746.99 lbs)
- `Plant density plants ha-1`: Very weak positive relationship with grain yield. The impact of plant density is minimal. Every 1 unit increase in this coef, grain yield should increase by 0.03 Kg / h1 units (~.07 lbs) 
- `Max_LAI`: Strong positive relationship with grain yield. This feature has a significant impact on grain yield. For every 1 unit increase in max leaf area, grain yield should increase by 527.54 Kg / h1 units (~1,163.03)

_Note: 1 hectare = 2.47 acres, or 107,639 square feet_

In [51]:
# Get feature names
feature_names = X.columns

# Define features as `X` and target variable (Grain Yield) as `y`
X = maize_model_data.drop(columns=['Grain Yield_15.5%mc_Kg ha-1'])
y = maize_model_data['Grain Yield_15.5%mc_Kg ha-1']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# initialize the model
linear_regression_model = LinearRegression()

# train the model on non-transformed data
linear_regression_model.fit(X_train, y_train)

# Extract and display the coefficients
coefs = pd.DataFrame(
linear_regression_model.coef_,
    columns=["Coefficients"],
    index=feature_names
)

coefs

,Coefficients
Trt_code,-338.829080
Plant density plants ha-1,0.036397
Max_LAI,527.542289


### Feature importance
Treatment code (watering) is most important while plant density is the least important

In [ ]:
# # get feature importance from Random Forest model
# feature_importances = random_forest_model.feature_importances_
# features = X.columns

# # Create a DataFrame to display feature importances
# importance_df = pd.DataFrame({
#     'Feature': features,
#     'Importance': feature_importances
# }).sort_values(by='Importance', ascending=False)

# importance_df